## Steps for processing

### Import packages used in analysis

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import geemap
# geemap.update_package()
import ee
ee.Initialize()

In [2]:
#function to plot dictionaries in python with dropdowns
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### Create a map Object to visualise spatial rasters and vectors

In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [4]:
countries = ee.FeatureCollection("USDOS/LSIB/2017")
geometry = ee.Geometry.Point([27.7,-25.5])
SA = countries.filterBounds(geometry).geometry()
# SA = ee.Feature(countries.filterBounds(geometry).geometry()).buffer(100000).geometry()
# Map.addLayer(SA)

### 1. Prepare WH Presence localities

### 2. Prepare WH Absence localities

### 3. Define GEE covariates to extract

#### List of Covariates that will be used:
1. Minimum temperature/ night temperature
2. Water seasonality
3. Precipitation
4. Distance to the coast
5. Elevation
6. CHILI (a surrogate for effects of insolation and topographic shading)
7. Global Human Modification
8. Topographic Diversity (a surrogate variable that represents the variety of temperature and moisture conditions available to species as local habitats)
9. Area of different landcover within 1km buffer
10. EVI ( a proxy of vegetation health)
11. water Nutrient levels
12. Competition (IAAP/Site Area)
13. Recuitment Suitability (area of shallow water)
14. Connectivity (Zoochory)
15. Connectivity (Hydrochory)
16. Connectivity (human aided dispersal)- distance from roads




#### 1. Minimum Temperature (Celsius)

In [19]:
# minT = ee.ImageCollection("MODIS/006/MOD11A1").filterBounds(SA).select('LST_Night_1km')
minT = ee.Image("WORLDCLIM/V1/BIO").select(["bio06"]).multiply(0.1).rename('minT').clip(SA)
Map.addLayer(minT,{'min':-15,'max':20, 'palette':['440154', '481567', '482677', '453781', '404788', '39568C', '33638D', '2D708E', '287D8E', '238A8D', '1F968B', '20A387', '29AF7F', '3CBB75', '55C667', '73D055', '95D840', 'B8DE29', 'DCE319', 'FDE725']},'minT')

#### 2. Water Seasonality

In [25]:
#Water- seasonality
startDate = ee.Date('2013-01-01')
endDate = ee.Date('2013-12-31')
water = ee.Image('JRC/GSW1_2/GlobalSurfaceWater')
wSeasonality = water.select(['seasonality']).rename('wSeasonality')
Map.addLayer(wSeasonality,{},'seasonality')
# print(seasonality.getInfo())

#### 3. Precipitation

In [21]:
#Precipitation (~5km)

rain = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").select(['pr'])

totalPrecipitation = rain.filterDate(startDate,endDate).sum().rename('precipitation').clip(SA)
Map.addLayer(totalPrecipitation,{},'Precipitation')

#### 4. Distance to Coast

In [23]:
#distance from the coast
D2Coast = ee.Image('users/zandersamuel/Global_misc/GMT_intermediate_coast_distance_01d').rename('distance_to_coast').clip(SA)
Map.addLayer(D2Coast,{},'distance from coast')

#### 5. Elevation

In [8]:
#elevation
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation').clip(SA)
Map.addLayer(elevation,{},'elevation')
# ee.Image("MERIT/DEM/v1_0_3")

#### 6. CHILI

In [9]:
chili = ee.Image('CSP/ERGo/1_0/Global/SRTM_CHILI').rename('CHILI').clip(SA)
Map.addLayer(chili,{},'CHILI')

#### 7. Global Human Modififcation

In [10]:
ghm = ee.ImageCollection('CSP/HM/GlobalHumanModification')
GHM = ghm.mosaic().rename('Global_Human_Modification').clip(SA)
Map.addLayer(GHM,{},'Global Human Modification')

#### 8. Topographic Diversity

In [11]:
topD = ee.Image('CSP/ERGo/1_0/Global/SRTM_topoDiversity').rename('topD').clip(SA)
Map.addLayer(topD,{},'topographic diversity')

#### 9. Frost duration and Frost degree days

In [26]:
frost = ee.Image('users/geethensingh/frostdur').unmask().clip(SA).rename('frost')
Map.addLayer(frost,{},'frost')

In [28]:
#number of days with a temperature less than 0 degrees celsius in 2013

modis = ee.ImageCollection("MODIS/006/MOD11A1").filter(ee.Filter.calendarRange(2013,2013,'year')).select('LST_Night_1km').filterBounds(SA)    
# print(modis.sort('system:time_start',false).first())

def islt0 (img):
  lt0 = img.multiply(0.02).subtract(273.15).lt(0)
  return lt0

FDs = modis.map(islt0).reduce(ee.Reducer.count()).clip(SA).rename('Tlt0')
Map.addLayer(FDs,{},'FDs');

In [29]:
# test_s = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global").select('.*coverfraction').getRegion(EG_CR_sample.first().geometry(),100).getInfo()
# print(test_s[:5])

#### 10. Number of days with temperature less than 10 dgrees

In [30]:
#number of days with a temperature less than 10 degrees celsius in 2013

modis = ee.ImageCollection("MODIS/006/MOD11A1").filter(ee.Filter.calendarRange(2013,2013,'year')).select('LST_Night_1km').filterBounds(SA)    
# print(modis.sort('system:time_start',false).first())

def islt10 (img):
  lt10 = img.multiply(0.02).subtract(273.15).lt(10)
  return lt10

FDs10 = modis.map(islt10).reduce(ee.Reducer.count()).rename('Tlt10')
# Map.addLayer(FDs10,{},'FDs');

## 4. Merge (non-Buffer) covariates as stack/multiband image

In [31]:
months = (['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])#
covariates = minT.addBands([wSeasonality, totalPrecipitation ,D2Coast,elevation,chili,GHM,topD, frost, FDs, FDs10])
print(covariates.bandNames().getInfo())
# Map.addLayer(covariates,{},'covariates')

['minT', 'wSeasonality', 'precipitation', 'distance_to_coast', 'elevation', 'CHILI', 'Global_Human_Modification', 'topD', 'frost', 'Tlt0', 'Tlt10']


## 5. Export stacked covariates to GEE Asset

In [32]:
task = ee.batch.Export.image.toAsset(**{'image':covariates,'description':'covs', 'assetId':'users/geethensingh/WH_covariates','region':SA.geometry(), 'scale':100, 'maxPixels':1e13})
task.start()

In [53]:
task.status()

{'state': 'COMPLETED',
 'description': 'covs',
 'creation_timestamp_ms': 1614327981177,
 'update_timestamp_ms': 1614330190564,
 'start_timestamp_ms': 1614327989642,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/earthengine-legacy/assets/users/geethensingh/WH_covariates'],
 'attempt': 1,
 'id': 'J65RAHPIKDE4TW3OTE2FLKRP',
 'name': 'projects/earthengine-legacy/operations/J65RAHPIKDE4TW3OTE2FLKRP'}

## 6. Read extracted covariates

#### 15. max duration of period less than 10

In [5]:
modis = ee.ImageCollection("MODIS/006/MOD11A1").filter(ee.Filter.calendarRange(2013,2013,'year')).select('LST_Night_1km').filterBounds(SA)    

def islt10 (img):
  lt10 = img.multiply(0.02).subtract(273.15).lt(10)
  return lt10

FDs10 = modis.map(islt10)#.reduce(ee.Reducer.count()).rename('Tlt10')

def consecutiveDays(this_img, cum_prev_max):
  cum_img = ee.Image(cum_prev_max).select(0) #//load cumulative # days
  prev_img = ee.Image(cum_prev_max).select(1) #//load prev day's image
  max_run = ee.Image(cum_prev_max).select(2) #//load maximum # consecutive data
  this_img = this_img.unmask() #//set masked pixels to 0
  cum_img = cum_img.where(this_img.eq(1).And(prev_img.eq(1)), cum_img.add(1)) #// if this and previous day were > 50, record this consecutive day 
  cum_img = cum_img.where(this_img.neq(1), 0) #//if < 50 deg, reset counter
  max_run = max_run.where(cum_img.gt(max_run),cum_img)
  return cum_img.addBands(this_img).addBands(max_run) #//this return value becomes cum_prev input

max_value = 40
T50plus = ee.ImageCollection(FDs10.toList(max_value))

max = ee.Image(T50plus.iterate(consecutiveDays, ee.Image([0,0,0]))).select(2).add(1).clip(SA) #// need to add 1 day to get # days rather than # two-date periods > 50 deg 
# print('max # consecutive days > 50 image',max.getInfo())
# Map.addLayer(max,{min: 0, max: max_value}, "max # consecutive days > 50")

# cum = ee.Image(T50plus.iterate(consecutiveDays, ee.Image([0,0,0]))).select(0).add(1).clip(SA)
# Map.addLayer(cum,{min: 0, max: max_value}, "cumulative # consecutive days > 50",False)

#### 12. Competition

In [6]:
WH13 = ee.Image("users/geethensingh/SA_hyacinth2013")
aq_veg = WH13.eq(0).selfMask().multiply(ee.Image.pixelArea()).rename('aq_veg_area') 
macro_veg = ee.Image.cat([aq_veg])
# print(macro_veg.bandNames().getInfo())

## 6. Read extracted covariates

In [7]:
covariates = ee.Image('users/geethensingh/WH_covariates').addBands([max.rename('maxDslt10'),macro_veg])

## 7. covs: part 2- feature vars

#### 13. Connectivity (hydrochory)

In [8]:
rvs = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers")
def connectivity(ft):
    reaches = rvs.filterBounds(ft.geometry())
    ft = ft.set('dsrcnt',reaches.filter(ee.Filter.gt('NDOID',0)).size())
    ft = ft.set('usrcnt',reaches.filter(ee.Filter.gt('NUOID',0)).size()) 
    return ft

###  Covariates (5km buffer): part 3

## 7. Split shapefile into portions, buffer and extract remaining variables

#### 11. Area of Landcover within buffer

In [9]:
bufferSize = 5000 #in metres

def buffer (feature):
    feature = feature.set('ft_area',feature.area(2))
    buffer_ft = feature.buffer(bufferSize)
    return buffer_ft

In [10]:
cats = ['contiguous (indigenous) forest',
 'contiguous low forest & thicket',
 'dense forest & woodland',
 'open woodland',
 'contiguous & dense plantation forest',
 'open & sparse plantation forest',
 'temporary unplanted (clear-felled) plantation forest',
 'low shrubland (other)',
 'low shrubland (fynbos)',
 'low shrubland (succulent karoo)',
 'low shrubland (nama karoo)',
 'sparsely wooded grassland',
 'natural grassland',
 'natural rivers',
 'natural estuaries & lagoons',
 'natural ocean & coastal',
 'natural lakes',
 'natural pans (flooded @ observation times)',
 'artificial dams (including canals)',
 'artificial sewage ponds',
 'artificial flooded mine pits',
 'herbaceous wetlands (currently mapped)',
 'herbaceous wetlands (previously mapped)',
 'mangrove wetlands',
 'natural rock surfaces',
 'dry pans',
 'eroded lands',
 'sand dunes (terrestrial)',
 'coastal sand & dunes',
 'bare riverbed material',
 'other bare',
 'cultivated commercial permanent orchards',
 'cultivated commercial permanent vines',
 'cultivated commercial sugarcane pivot irrigated',
 'cultivated commercial permanent pineapples',
 'cultivated commercial sugarcane non-pivot',
 'cultivated emerging farmer sugarcane non-pivot',
 'commercial annual crops pivot irrigated',
 'commercial annual crops non-pivot irrigated',
 'commercial annual crops rain-fed / dryland',
 'subsistence / small-scale annual crops',
 'fallow land & old fields (trees)',
 'fallow land & old fields (bush)',
 'fallow land & old fields (grass)',
 'fallow land & old fields (bare)',
 'fallow land & old fields (low shrub)',
 'residential formal (tree)',
 'residential formal (bush)',
 'residential formal (low veg / grass)',
 'residential formal (bare)',
 'residential informal (tree)',
 'residential informal (bush)',
 'residential informal (low veg / grass)',
 'residential informal (bare)',
 'village scattered (bare & low veg/ grss combo)',
 'village dense (bare & low veg / grss combo)',
 'smallholdings (tree)',
 'smallholdings (bush)',
 'smallholdings (low veg / grass)',
 'smallholdings (bare)',
 'urban recreational fields (tree)',
 'urban recreational fields (bush)',
 'urban recreational fields (grass)',
 'urban recreational fields (bare)',
 'commercial',
 'industrial',
 'roads & rails (major linear)',
 'mines: surface infrastructure',
 'mines: extraction pits, quarries',
 'mines: salt mines',
 'mine: tailings and resource dumps',
 'land-fills',
 'fallow land & old fields (wetlands)']

cats = ee.List(cats)

In [11]:
#functions to format dictionary from SANLC
def unpack(list_item):
    areaDict = ee.Dictionary(list_item)
    cat = cats.get(ee.Number(areaDict.get('b1')).subtract(1))
    area = ee.Number(areaDict.get('sum')).divide(1e6)
    return ee.List([cat, area])

def classAreas (ft):
    opt = ee.List(ft.get('groups')).map(unpack)
    return ee.Feature(ft.geometry(),ee.Dictionary(opt.flatten()));

#### Detailed LC (SANLC)

In [12]:
SANLC = ee.Image("users/geethensingh/SANLC__2013_14").rename('SANLC')

#### 14. Water Nutrients proxy (riparian N, SOC and phh20)

#### 16. Flooding (refer to 13 connectivity)

In [13]:
LC = ee.Image(ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global").first().clip(SA)).select('.*coverfraction')
N = ee.Image('projects/soilgrids-isric/nitrogen_mean')
SOC = ee.Image("projects/soilgrids-isric/soc_mean")
phh20 = ee.Image("projects/soilgrids-isric/phh2o_mean")
flood = ee.Image("users/geethensingh/floodMapGL_rp10y").rename('flood').clip(SA)
mnnlrnff = ee.Image("users/geethensingh/SA_MAR").rename('runoff')
buffer_cov = ee.Image.cat([LC, N,SOC,phh20,flood, mnnlrnff])
# print(buffer_cov.bandNames().getInfo())

In [14]:
spatialFilter = ee.Filter.intersects(**{
  'leftField': '.geo',
  'rightField': '.geo',
  'maxError': 5
});

simpleJoin = ee.Join.saveFirst(**{'matchKey':'matches'});

def formatter (f):
    f1 = ee.Feature(f)
    f2 = ee.Feature(f.get('matches')).toDictionary()
    return ee.Feature(f1).set(f2).set('matches', None)

# RenderJSON(result.getInfo())


In [15]:
grid = ee.FeatureCollection('users/geethensingh/SA_Grid_1dgr')
# // Define a save all join.
saveFirstJoin = ee.Join.saveFirst(**{'matchKey': 'matches',});

### Sanity check

In [34]:
#Sanity Check
shp_dir = r"D:\phd\Chapter_3\nat_drivers\1inputs\WH_wbs.shp"
shp = gpd.read_file(shp_dir, rows = 5)
ee_shp = geemap.geopandas_to_ee(shp)
vars1 = covariates.reduceRegions(**{'collection': ee_shp,'reducer': ee.Reducer.mean(),'scale': 30, 'tileScale': 8})
# RenderJSON(buffer_covs2.getInfo())
                #extracts part 2 vars: feature
ft_cov = vars1.map(connectivity)
# Get grid ID.
ft_cov = saveFirstJoin.apply(ft_cov, grid, spatialFilter).map(formatter)
# RenderJSON(vars1.getInfo())
                #extract part 3 vars: ft with buffer SANLC
buffer_poly = ft_cov.map(buffer)
buffer_covs1 = (ee.Image.pixelArea().addBands(SANLC).reduceRegions(**{'collection': buffer_poly,
                              'reducer': ee.Reducer.sum().group(**{'groupField': 1, 'groupName': 'b1'}),
                                       'scale': 30, 'tileScale': 8}).map(classAreas))
              #extract part 3 vars: ft with buffer
buffer_covs2 = (buffer_cov.reduceRegions(**{'collection' : buffer_covs1, 'reducer':ee.Reducer.mean()
                                                     ,'scale':30, 'tileScale' : 8}))
# RenderJSON(buffer_cov.getInfo())
fc_joined = simpleJoin.apply(ft_cov, buffer_covs2, spatialFilter);
result = (fc_joined.map(formatter))
RenderJSON(result.getInfo())

In [93]:
shp_dir = r"D:\phd\Chapter_3\nat_drivers\1inputs\WH_wbs.shp"
shp = gpd.read_file(shp_dir)

In [94]:
int(len(shp))

248370

In [113]:
def shp_job_splitter(shp, features_per_split, GD_folder_name):
    shp=shp
    tsks=1
    for i in range(0,len(shp), features_per_split):
        if tsks >3000-features_per_split:
            input('click enter')
            tsks = 1
        if i+features_per_split > len(shp):
            data = np.split(shp, range(i, len(shp) , features_per_split))#list of lists
            print(i)
            tsks += features_per_split
        else:
            data = shp.iloc[i:i+features_per_split,:]
            print(len(data))
           #gpddf = gpd.GeoDataFrame(data)
            ee_shp = geemap.geopandas_to_ee(data)
            task_name = 'Export_batch'+ str(int(round(i/features_per_split+1)))
            print('Executing...'+ task_name)
            tsks += features_per_split

In [114]:
shp_job_splitter(shp, 30, 'Earth_Engine')

30
Executing...Export_batch1
30
Executing...Export_batch2
30
Executing...Export_batch3
30
Executing...Export_batch4
30
Executing...Export_batch5
30
Executing...Export_batch6
30
Executing...Export_batch7
30
Executing...Export_batch8
30
Executing...Export_batch9
30
Executing...Export_batch10
30
Executing...Export_batch11
30
Executing...Export_batch12
30
Executing...Export_batch13
30
Executing...Export_batch14
30
Executing...Export_batch15
30
Executing...Export_batch16
30
Executing...Export_batch17
30
Executing...Export_batch18
30
Executing...Export_batch19
30
Executing...Export_batch20
30
Executing...Export_batch21
30
Executing...Export_batch22
30
Executing...Export_batch23
30
Executing...Export_batch24
30
Executing...Export_batch25
30
Executing...Export_batch26
30
Executing...Export_batch27
30
Executing...Export_batch28
30
Executing...Export_batch29
30
Executing...Export_batch30
30
Executing...Export_batch31
30
Executing...Export_batch32
30
Executing...Export_batch33
30
Executing...Expo

KeyboardInterrupt: 

## Step 3. Extract Covariates

In [16]:
def shp_job_splitter(shp_dir, features_per_split, GD_folder_name):
    shp = gpd.read_file(shp_dir)
    tsks=1
    for i in range(5996*35,len(shp), features_per_split):
        if tsks >3000-2:
            input("Click here once previous 2k(x) tasks have completed")
            tsks = 1
        if i+features_per_split > len(shp):
            data = shp.iloc[i:len(shp),:]
            ee_shp = geemap.geopandas_to_ee(data)
            #extracts part 1 vars
            vars1 = covariates.reduceRegions(**{'collection': ee_shp,'reducer': ee.Reducer.mean(),'scale': 30, 'tileScale': 8})
            #extracts part 2 vars: feature
            ft_cov = vars1.map(connectivity)
            # Get grid ID.
            ft_cov = saveFirstJoin.apply(ft_cov, grid, spatialFilter).map(formatter)
            #extract part 3 vars: ft with buffer SANLC
            buffer_poly = ft_cov.map(buffer)
            buffer_covs1 = (ee.Image.pixelArea().addBands(SANLC).reduceRegions(**{'collection': buffer_poly,
                                              'reducer': ee.Reducer.sum().group(**{'groupField': 1, 'groupName': 'b1'}),
                                                       'scale': 30, 'tileScale': 8}).map(classAreas))
            #extract part 3 vars: ft with buffer
            buffer_covs2 = (buffer_cov.reduceRegions(**{'collection' : buffer_covs1, 'reducer':ee.Reducer.mean()
                                                                     ,'scale':30, 'tileScale' : 8}))
            # RenderJSON(buffer_cov.getInfo())
            fc_joined = simpleJoin.apply(ft_cov, buffer_covs2, spatialFilter);
            result = (fc_joined.map(formatter))
            task_name = 'Export_batch' + str(int(round(i/features_per_split+1)))
            print('Executing...'+ task_name)
            task = ee.batch.Export.table.toDrive(result, task_name, GD_folder_name)
            task.start()
            tsks += 1
        else:
            data = shp.iloc[i:i+features_per_split,:]
            ee_shp = geemap.geopandas_to_ee(data)
            #extracts part 1 vars
            vars1 = covariates.reduceRegions(**{'collection': ee_shp,'reducer': ee.Reducer.mean(),'scale': 30, 'tileScale': 8})
            #extracts part 2 vars: feature
            ft_cov = vars1.map(connectivity)
            # Get grid ID.
            ft_cov = saveFirstJoin.apply(ft_cov, grid, spatialFilter).map(formatter)
            #extract part 3 vars: ft with buffer SANLC
            buffer_poly = ft_cov.map(buffer)
            buffer_covs1 = (ee.Image.pixelArea().addBands(SANLC).reduceRegions(**{'collection': buffer_poly,
                                              'reducer': ee.Reducer.sum().group(**{'groupField': 1, 'groupName': 'b1'}),
                                                       'scale': 30, 'tileScale': 8}).map(classAreas))
            #extract part 3 vars: ft with buffer
            buffer_covs2 = (buffer_cov.reduceRegions(**{'collection' : buffer_covs1, 'reducer':ee.Reducer.mean()
                                                                     ,'scale':30, 'tileScale' : 8}))
            # RenderJSON(buffer_cov.getInfo())
            fc_joined = simpleJoin.apply(ft_cov, buffer_covs2, spatialFilter);
            result = (fc_joined.map(formatter))
            task_name = 'Export_batch_' + str(int(round(i/features_per_split+1)))
            print('Executing...'+ task_name)
            task = ee.batch.Export.table.toDrive(result, task_name, GD_folder_name)
            task.start()
            tsks += 1

In [17]:
shp_dir = r"D:\phd\Chapter_3\nat_drivers\1inputs\WH_wbs.shp"
shp_job_splitter(shp_dir, 35, 'Earth_Engine')

Executing...Export_batch_5997
Executing...Export_batch_5998
Executing...Export_batch_5999
Executing...Export_batch_6000
Executing...Export_batch_6001
Executing...Export_batch_6002
Executing...Export_batch_6003
Executing...Export_batch_6004
Executing...Export_batch_6005
Executing...Export_batch_6006
Executing...Export_batch_6007
Executing...Export_batch_6008
Executing...Export_batch_6009
Executing...Export_batch_6010
Executing...Export_batch_6011
Executing...Export_batch_6012
Executing...Export_batch_6013
Executing...Export_batch_6014
Executing...Export_batch_6015
Executing...Export_batch_6016
Executing...Export_batch_6017
Executing...Export_batch_6018
Executing...Export_batch_6019
Executing...Export_batch_6020
Executing...Export_batch_6021
Executing...Export_batch_6022
Executing...Export_batch_6023
Executing...Export_batch_6024
Executing...Export_batch_6025
Executing...Export_batch_6026
Executing...Export_batch_6027
Executing...Export_batch_6028
Executing...Export_batch_6029
Executing.

Executing...Export_batch_6271
Executing...Export_batch_6272
Executing...Export_batch_6273
Executing...Export_batch_6274
Executing...Export_batch_6275
Executing...Export_batch_6276
Executing...Export_batch_6277
Executing...Export_batch_6278
Executing...Export_batch_6279
Executing...Export_batch_6280
Executing...Export_batch_6281
Executing...Export_batch_6282
Executing...Export_batch_6283
Executing...Export_batch_6284
Executing...Export_batch_6285
Executing...Export_batch_6286
Executing...Export_batch_6287
Executing...Export_batch_6288
Executing...Export_batch_6289
Executing...Export_batch_6290
Executing...Export_batch_6291
Executing...Export_batch_6292
Executing...Export_batch_6293
Executing...Export_batch_6294
Executing...Export_batch_6295
Executing...Export_batch_6296
Executing...Export_batch_6297
Executing...Export_batch_6298
Executing...Export_batch_6299
Executing...Export_batch_6300
Executing...Export_batch_6301
Executing...Export_batch_6302
Executing...Export_batch_6303
Executing.

Executing...Export_batch_6545
Executing...Export_batch_6546
Executing...Export_batch_6547
Executing...Export_batch_6548
Executing...Export_batch_6549
Executing...Export_batch_6550
Executing...Export_batch_6551
Executing...Export_batch_6552
Executing...Export_batch_6553
Executing...Export_batch_6554
Executing...Export_batch_6555
Executing...Export_batch_6556
Executing...Export_batch_6557
Executing...Export_batch_6558
Executing...Export_batch_6559
Executing...Export_batch_6560
Executing...Export_batch_6561
Executing...Export_batch_6562
Executing...Export_batch_6563
Executing...Export_batch_6564
Executing...Export_batch_6565
Executing...Export_batch_6566
Executing...Export_batch_6567
Executing...Export_batch_6568
Executing...Export_batch_6569
Executing...Export_batch_6570
Executing...Export_batch_6571
Executing...Export_batch_6572
Executing...Export_batch_6573
Executing...Export_batch_6574
Executing...Export_batch_6575
Executing...Export_batch_6576
Executing...Export_batch_6577
Executing.

Executing...Export_batch_6819
Executing...Export_batch_6820
Executing...Export_batch_6821
Executing...Export_batch_6822
Executing...Export_batch_6823
Executing...Export_batch_6824
Executing...Export_batch_6825
Executing...Export_batch_6826
Executing...Export_batch_6827
Executing...Export_batch_6828
Executing...Export_batch_6829
Executing...Export_batch_6830
Executing...Export_batch_6831
Executing...Export_batch_6832
Executing...Export_batch_6833
Executing...Export_batch_6834
Executing...Export_batch_6835
Executing...Export_batch_6836
Executing...Export_batch_6837
Executing...Export_batch_6838
Executing...Export_batch_6839
Executing...Export_batch_6840
Executing...Export_batch_6841
Executing...Export_batch_6842
Executing...Export_batch_6843
Executing...Export_batch_6844
Executing...Export_batch_6845
Executing...Export_batch_6846
Executing...Export_batch_6847
Executing...Export_batch_6848
Executing...Export_batch_6849
Executing...Export_batch_6850
Executing...Export_batch_6851
Executing.

Executing...Export_batch_7093
Executing...Export_batch_7094
Executing...Export_batch_7095
Executing...Export_batch_7096
Executing...Export_batch7097
